# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on April 13 (with 2-hour grace period) and worth 25 points

### Overview

This milestone builds on your work from Milestone 1. You will:

1. Evaluate baseline models using default settings.
2. Engineer new features and re-evaluate models.
3. Use feature selection techniques to find promising subsets.
4. Select the top 3 models and fine-tune them for optimal performance.

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone.


In [49]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV,
    RandomizedSearchCV,
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



## Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling.

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>   df = pd.read_csv("zillow_cleaned.csv")

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:**

- You will not use the testing set during this milestone — it’s reserved for final evaluation later.
- You will have to redo the scaling step when you introduce new features (which have to be scaled as well).


In [10]:
# Load the CSV file
df = pd.read_csv("zillow_cleaned.csv")

# Display the first few rows of the DataFrame
print(df.head())


   bathroomcnt  bedroomcnt  buildingqualitytypeid  calculatedbathnbr  \
0          3.5         4.0                    6.0                3.5   
1          1.0         2.0                    6.0                1.0   
2          2.0         3.0                    6.0                2.0   
3          3.0         4.0                    8.0                3.0   
4          3.0         3.0                    8.0                3.0   

   calculatedfinishedsquarefeet  finishedsquarefeet12    fips  fullbathcnt  \
0                        3100.0                3100.0  6059.0          3.0   
1                        1465.0                1465.0  6111.0          1.0   
2                        1243.0                1243.0  6059.0          2.0   
3                        2376.0                2376.0  6037.0          3.0   
4                        1312.0                1312.0  6037.0          3.0   

   heatingorsystemtypeid    latitude  ...  propertyzoningdesc  \
0                    2.0  3363493

In [ ]:
# ** PLEASE NOTE ***
# This assignment can also be viewed: 
#       GitHub: https://github.com/Siobhan-C/Milestone_Two/tree/main
#       Google Drive / Google Colab: https://drive.google.com/drive/folders/1S5QyGJYHvvJ0CZrOL7dLQ11-mJ_v2-cZ?usp=sharing (see Milestone_02_Final)

# Add as many code cells as you need
#Laura:This is on my own drive, not sure how to edit it on the cloud
#Laura: df = pd.read_csv(r"C:\Users\laurr\OneDrive\Desktop\Boston University\Module 3\Milestone 1\zillow_cleaned.csv")
#Emmanuel: df = pd.read_csv('zillow_cleaned.csv')

#Siobhan: The previous and following comments explain the process of loading the cleaned dataset (zillow_cleaned) from Milestone 1 into the project, as we faced challenges with accessing the shared folder.
#Initially, we decided to store the cleaned dataset from Milestone 1 (zillow_cleaned) within the shared folder on Google Drive, which can be accessed via the following link: https://drive.google.com/file/d/1f-4CrmrAH2JJy51ei1Aq_kaYGPI3S_gn/view?usp=share_link

#The code to generate the CSV file from the cleaned DataFrame:
#df_cleaned.to_csv("zillow_cleaned.csv", index=False)
## For downloading the file:
#from google.colab import files
#files.download('zillow_cleaned.csv')
#Read the file into a DataFrame:
#df = pd.read_csv("zillow_cleaned.csv")

#While the zillow_cleaned dataset was initially uploaded to our shared folder, I (Siobhan) faced difficulties importing the data directly from the shared folder due to Colab's limitations with shared folder access.
#Due to these issues, I opted to use VS code.
# For convenience, here is the link to the shared folder containing the zillow_cleaned.csv file and other milestone content: https://drive.google.com/drive/folders/1S5QyGJYHvvJ0CZrOL7dLQ11-mJ_v2-cZ?usp=share_link


#load zillow cleaned CSV file from milestone 1 into DF
df = pd.read_csv("zillow_cleaned.csv")


In [12]:
#show few rows of the DataFrame
df.head()

,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,latitude,...,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,yearbuilt,censustractandblock,taxvaluedollarcnt,log_taxvaluedollarcnt
0,3.5,4.0,6.0,3.5,3100.0,3100.0,6059.0,3.0,2.0,33634931.0,...,564.0,6.059063e+07,53571.0,1286.0,96978.0,0.0,1998.0,6.059063e+13,1023282.0,13.838527
1,1.0,2.0,6.0,1.0,1465.0,1465.0,6111.0,1.0,2.0,34449266.0,...,564.0,6.111001e+07,13091.0,2061.0,97099.0,5.0,1967.0,6.111001e+13,464000.0,13.047642
2,2.0,3.0,6.0,2.0,1243.0,1243.0,6059.0,2.0,2.0,33886168.0,...,564.0,6.059022e+07,21412.0,1286.0,97078.0,6.0,1962.0,6.059022e+13,564778.0,13.244190
3,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,2.0,34245180.0,...,767.0,6.037300e+07,396551.0,3101.0,96330.0,0.0,1970.0,6.037300e+13,145143.0,11.885482
4,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,3.0,2.0,34185120.0,...,571.0,6.037124e+07,12447.0,3101.0,96451.0,0.0,1964.0,6.037124e+13,119407.0,11.690301


In [13]:
#show few rows of the DataFrame
print(df.head())


   bathroomcnt  bedroomcnt  buildingqualitytypeid  calculatedbathnbr  \
0          3.5         4.0                    6.0                3.5   
1          1.0         2.0                    6.0                1.0   
2          2.0         3.0                    6.0                2.0   
3          3.0         4.0                    8.0                3.0   
4          3.0         3.0                    8.0                3.0   

   calculatedfinishedsquarefeet  finishedsquarefeet12    fips  fullbathcnt  \
0                        3100.0                3100.0  6059.0          3.0   
1                        1465.0                1465.0  6111.0          1.0   
2                        1243.0                1243.0  6059.0          2.0   
3                        2376.0                2376.0  6037.0          3.0   
4                        1312.0                1312.0  6037.0          3.0   

   heatingorsystemtypeid    latitude  ...  propertyzoningdesc  \
0                    2.0  3363493

In [14]:
#Train/Test Split - Laura
X = df.drop(columns=["taxvaluedollarcnt"])
y = df["taxvaluedollarcnt"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
#Standardizing features on training data - Laura
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

### Part 1: Baseline Modeling [3 pts]

Apply the following regression models to the scaled training dataset using **default parameters**:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each model:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV RMSE Score** across all folds in a table.


In [16]:
# Add as many code cells as you need - Laura
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score, RepeatedKFold
#custom RMSE function
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
#make_scorer used to convert custom RMSE function into a form that cross_val_score and other model evaluation tools can use
rmse_scorer = make_scorer(rmse, greater_is_better=False)
#sets up repeated kfold cross validation
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

In [17]:
#Linear Regression - Laura
lr_model = LinearRegression()
scores_lr = cross_val_score(lr_model, X_train_scaled, y_train, scoring=rmse_scorer, cv=cv)
print("Linear Regression - Mean RMSE:", -np.mean(scores_lr))
print("Linear Regression - Std RMSE:", np.std(scores_lr))

Linear Regression - Mean RMSE: 121790.89257365465
Linear Regression - Std RMSE: 71236.18280439186


In [18]:
#RIdge -  Laura
ridge_model = Ridge()
scores_ridge = cross_val_score(ridge_model, X_train_scaled, y_train, scoring=rmse_scorer, cv=cv)
print("Ridge Regression - Mean RMSE:", -np.mean(scores_ridge))
print("Ridge Regression - Std RMSE:", np.std(scores_ridge))

Ridge Regression - Mean RMSE: 105429.22202740233
Ridge Regression - Std RMSE: 33737.24364490555


In [19]:
#Lasso - Laura
lasso_model = Lasso()
scores_lasso = cross_val_score(lasso_model, X_train_scaled, y_train, scoring=rmse_scorer, cv=cv)
print("Lasso Regression - Mean RMSE:", -np.mean(scores_lasso))
print("Lasso Regression - Std RMSE:", np.std(scores_lasso))

/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.577e+14, tolerance: 2.565e+11
  model = cd_fast.enet_coordinate_descent(
/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.619e+14, tolerance: 2.552e+11
  model = cd_fast.enet_coordinate_descent(
/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

Lasso Regression - Mean RMSE: 108011.62058739505
Lasso Regression - Std RMSE: 41611.18840041595


/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.616e+14, tolerance: 2.555e+11
  model = cd_fast.enet_coordinate_descent(


In [20]:
#DecisionTree - Laura
from sklearn.tree import DecisionTreeRegressor
tree_model = DecisionTreeRegressor()
scores_tree = cross_val_score(tree_model, X_train_scaled, y_train, scoring=rmse_scorer, cv=cv)
print("Decision Tree - Mean RMSE:", -np.mean(scores_tree))
print("Decision Tree - Std RMSE:", np.std(scores_tree))

Decision Tree - Mean RMSE: 83.83762360146673
Decision Tree - Std RMSE: 25.01851207115175


In [21]:
#Bagging - Laura
bagging_model = BaggingRegressor()
scores_bagging = cross_val_score(bagging_model, X_train_scaled, y_train, scoring=rmse_scorer, cv=cv)
print("Bagging - Mean RMSE:", -np.mean(scores_bagging))
print("Bagging - Std RMSE:", np.std(scores_bagging))

Bagging - Mean RMSE: 66.87087747680911
Bagging - Std RMSE: 24.67604570572015


In [22]:
#RandomForest -  Laura
rf_model = RandomForestRegressor()
scores_rf = cross_val_score(rf_model, X_train_scaled, y_train, scoring=rmse_scorer, cv=cv)
print("Random Forest - Mean RMSE:", -np.mean(scores_rf))
print("Random Forest - Std RMSE:", np.std(scores_rf))

Random Forest - Mean RMSE: 57.56411796289493
Random Forest - Std RMSE: 25.527431589459578


In [23]:
#GradientBoosting - Laura
gb_model = GradientBoostingRegressor()
scores_gb = cross_val_score(gb_model, X_train_scaled, y_train, scoring=rmse_scorer, cv=cv)
print("Gradient Boosting - Mean RMSE:", -np.mean(scores_gb))
print("Gradient Boosting - Std RMSE:", np.std(scores_gb))

Gradient Boosting - Mean RMSE: 1929.9621000356478
Gradient Boosting - Std RMSE: 17.943048162505537


### Part 1: Discussion [2 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which models perform best overall?
  - Which are most stable (lowest std)?
  - Any signs of overfitting or underfitting?

### Best Performing Models (Lowest Mean RMSE):
- The Random Forest model performed best overall with the lowest mean RMSE (~57), followed by Bagging ( ~66) and Decision Tree ( ~84).
- These models clearly outperformed the linear models (Linear, Ridge, Lasso), which had much higher RMSE values (over 100,000).

### Most Stable Models (Lowest Standard Deviation):
- Gradient Boosting had the lowest standard deviation (~18), indicating very consistent performance across cross-validation folds.
- Decision Tree and Random Forest also showed reasonable stability with standard deviations around 22–25.

### Signs of Overfitting or Underfitting:
- The linear models (Linear, Ridge, Lasso) had extremely high RMSE values compared to the tree-based models. This suggests underfitting, likely because they cannot capture the complex patterns in the data.
- Decision Tree had a very low RMSE (~84), but lower than expected standard deviation, which might indicate slight overfitting especially since ensemble methods (Bagging/Random Forest) perform better by reducing variance.
- The Lasso Regression produced convergence warnings, meaning it struggled to find an optimal solution. This could be due to needing more iterations or stronger regularization.


### Part 2: Feature Engineering [3 pts]

Consider **at least three new features** based on your Milestone 1, Part 5. Examples include:
- Polynomial terms
- Log or interaction terms
- Groupings or transformations of categorical features

Add these features to `X_train` and then:
- Scale using `StandardScaler`
- Re-run all models listed above (using default settings again).
- Report updated RMSE scores (mean and std) across repeated CV in a table.

**Note:**  Recall that this will require creating a new version of the dataset, so effectively you may be running "polynomial regression" using `LinearRegression`.

In [24]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
#rest of your code
df_fe = df.copy()

# New features
df_fe["bed_bath_interaction"] = df_fe["bedroomcnt"] * df_fe["bathroomcnt"]
df_fe["rooms_per_bedroom"] = df_fe["roomcnt"] / (df_fe["bedroomcnt"] + 1)
df_fe["property_age"] = 2024 - df_fe["yearbuilt"]

X_fe = df_fe.drop(columns=["taxvaluedollarcnt", "log_taxvaluedollarcnt"])
y_fe = df_fe["taxvaluedollarcnt"]

X_train_fe, X_test_fe, y_train_fe, y_test_fe = train_test_split(X_fe, y_fe, test_size=0.2, random_state=42)

# Standardize all features
scaler_fe = StandardScaler()
X_train_fe_scaled = scaler_fe.fit_transform(X_train_fe)

In [28]:
df_fe = df.copy()

# New features
df_fe["bed_bath_interaction"] = df_fe["bedroomcnt"] * df_fe["bathroomcnt"]
df_fe["rooms_per_bedroom"] = df_fe["roomcnt"] / (df_fe["bedroomcnt"] + 1)
df_fe["property_age"] = 2024 - df_fe["yearbuilt"]

X_fe = df_fe.drop(columns=["taxvaluedollarcnt", "log_taxvaluedollarcnt"])
y_fe = df_fe["taxvaluedollarcnt"]

X_train_fe, X_test_fe, y_train_fe, y_test_fe = train_test_split(X_fe, y_fe, test_size=0.2, random_state=42)

# Standardize all features
scaler_fe = StandardScaler()
X_train_fe_scaled = scaler_fe.fit_transform(X_train_fe)

In [29]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Decision Tree": DecisionTreeRegressor(),
    "Bagging": BaggingRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor()
}

results = []

for name, model in models.items():
    scores = cross_val_score(model, X_train_fe_scaled, y_train_fe, scoring=rmse_scorer, cv=cv)
    results.append({
        "Model": name,
        "Mean RMSE": -np.mean(scores),
        "Std RMSE": np.std(scores)
    })


/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.593e+14, tolerance: 2.565e+11
  model = cd_fast.enet_coordinate_descent(
/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.566e+14, tolerance: 2.552e+11
  model = cd_fast.enet_coordinate_descent(
/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

In [30]:
results_df = pd.DataFrame(results)
results_df

,Model,Mean RMSE,Std RMSE
0,Linear Regression,234827.408842,86829.268992
1,Ridge Regression,233922.225937,80257.102580
2,Lasso Regression,235926.608108,85708.527905
3,Decision Tree,244826.042666,1665.688326
4,Bagging,186185.470183,1329.377037
5,Random Forest,179027.752196,1466.436968
6,Gradient Boosting,179011.597651,1281.653776


### Part 2: Discussion [2 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

- Which new features seemed to help — and in which models?

- Do you have any hypotheses about why a particular feature helped (or didn’t)?

- Were there any unexpected results?



### Did any models show notable improvement in performance?

No, none of the models showed notable improvement in performance after adding the new features. In fact, the models performed worse, with increased RMSE compared to earlier runs.

### Which new features seemed to help — and in which models?

The new features included: bed_bath_interaction, rooms_per_bedroom, property_age (derived from yearbuilt)

Among these, none clearly improved performance.

### Do you have any hypotheses about why a particular feature helped (or didn’t)?

Interaction features like bed_bath_interaction may have added unnecessary complexity to linear models.

rooms_per_bedroom and property_age may not have strong predictive power or may be too noisy across different home layouts to meaningfully impact predictions.

### Were there any unexpected results?

Yes, it was surprising that linear models consistently performed worse.

### Part 3: Feature Selection [3 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features.
- Report updated RMSE scores (mean and std) across repeated CV in a table.


In [31]:
X_full = df_fe.drop(columns=["taxvaluedollarcnt", "log_taxvaluedollarcnt"])
y_full = df_fe["taxvaluedollarcnt"]

In [32]:
rf_selector = RandomForestRegressor(random_state=42)
rf_selector.fit(X_full, y_full)

importances = rf_selector.feature_importances_
feature_names = X_full.columns
selected_features = feature_names[importances > np.mean(importances)]

print("Selected Features:", selected_features.tolist())

X_selected = X_full[selected_features]

X_train_sel, X_test_sel, y_train_sel, y_test_sel = train_test_split(X_selected, y_full, test_size=0.2, random_state=42)
scaler_sel = StandardScaler()
X_train_sel_scaled = scaler_sel.fit_transform(X_train_sel)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)


#same as in P2
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Decision Tree": DecisionTreeRegressor(),
    "Bagging": BaggingRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor()
}


cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)


Selected Features: ['calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'latitude', 'longitude', 'lotsizesquarefeet', 'regionidzip', 'yearbuilt', 'property_age']


In [33]:
results_selected = []

for name, model in models.items():
        scores = cross_val_score(model, X_train_sel_scaled, y_train_sel, scoring=rmse_scorer, cv=cv)
        results_selected.append({
            "Model": name,
            "Mean RMSE": -np.mean(scores),
            "Std RMSE": np.std(scores)
        })


results_selected_df = pd.DataFrame(results_selected)
display(results_selected_df)

,Model,Mean RMSE,Std RMSE
0,Linear Regression,197492.951025,1159.593359
1,Ridge Regression,197492.948561,1159.600009
2,Lasso Regression,197492.945589,1159.608723
3,Decision Tree,245113.106543,2005.211004
4,Bagging,187358.016780,1542.256557
5,Random Forest,180220.573116,1586.321542
6,Gradient Boosting,180029.555699,1204.860960


### Part 3: Discussion [2 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?

- Which features were consistently retained across models?

- Were any of your newly engineered features selected as important?

- How did feature selection differ between linear and tree-based models?

**Did performance improve for any models after reducing the number of features?**

No, performance did not improve after feature selection. In fact, all models either maintained similar RMSE values (as seen with the linear models) or showed a slight decrease in performance.

**Which features were consistently retained across models?**

Since we used Random Forest feature importances to guide feature selection, the retained features were those considered most predictive by the Random Forest model. These are: calculatedfinishedsquarefeet, finishedsquarefeet12, latitude, longitude, lotsizesquarefeet, regionidzip, yearbuilt.

**Were any of your newly engineered features selected as important?**

Sadly no.

**How did feature selection differ between linear and tree-based models?**

We used Random Forest to pick features, so every model got the same smaller
set. But since those features were chosen based on what works best for trees (like non-linear patterns), it probably helped the tree-based models more.

### Part 4: Fine-Tuning Your Top 3 Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far.

1. Choose the top 3 models based on performance and interpretability from earlier parts.
2. For each model:
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, or other techniques from previous homeworks.
   - Experiment with different versions of your feature engineering and preprocessing — treat these as additional tunable components.
3. Report the mean and standard deviation of CV RMSE score for each model in a summary table.



In [35]:
# Add as many code cells as you need
#mean and standard deviation of CV RMSE score for each model in a summary table prior to tuning
results_selected_df = pd.DataFrame(results_selected)
display(results_selected_df)

,Model,Mean RMSE,Std RMSE
0,Linear Regression,197492.951025,1159.593359
1,Ridge Regression,197492.948561,1159.600009
2,Lasso Regression,197492.945589,1159.608723
3,Decision Tree,245113.106543,2005.211004
4,Bagging,187358.016780,1542.256557
5,Random Forest,180220.573116,1586.321542
6,Gradient Boosting,180029.555699,1204.860960


In [59]:
#import necessary libraries -- importing again, had issues with colab vs. vs code
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

In [62]:
#define models / hyperparameter grids for those selected
models = {
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "Ridge": Ridge()
}

param_grids = {
    "GradientBoostingRegressor": {
        "model__n_estimators": [50, 100, 150],
        "model__learning_rate": [0.01, 0.05, 0.1],
        "model__max_depth": [3, 5, 7],
    },
    "RandomForestRegressor": {
        "model__n_estimators": [100, 200],
        "model__max_depth": [10, 20, None],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2],
    },
    "Ridge": {
        "model__alpha": [0.1, 1, 10],
    }
}

#preprocessing pipeline — scaling, imputation, feature selection, polynomial features
def create_pipeline(model_name, k=10):
    """
    Create a pipeline with preprocessing (scaling, imputation, polynomial features, feature selection) and the model.
    """
    if model_name == "Ridge":
        pipeline = Pipeline([
            ("scaler", StandardScaler()),  # Feature scaling
            ("poly", PolynomialFeatures(degree=2, include_bias=False)),  # Add polynomial features
            ("imputer", SimpleImputer(strategy="mean")),  # Handle missing data
            ("model", models[model_name])  # Model itself
        ])
    else:
        pipeline = Pipeline([
            ("imputer", SimpleImputer(strategy="mean")),  # Handle missing data
            ("feature_selection", SelectKBest(score_func=f_regression, k=k)),  # Select top k features
            ("model", models[model_name])  # Model itself
        ])
    
    return pipeline

#hyperparameter tuning - GridSearchCV
def tune_model(model_name, param_grid, k=10):
    """
    Perform hyperparameter tuning for the given model using GridSearchCV.
    """
    pipeline = create_pipeline(model_name, k)
    param_grid = param_grids[model_name]
    
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    
    return best_model

#fine-tune the selected models
best_models = {}
for model_name in ["GradientBoostingRegressor", "RandomForestRegressor", "Ridge"]:
    print(f"Tuning {model_name}...")
    best_models[model_name] = tune_model(model_name, param_grids, k=10)  # Add feature selection with k=10

#eval / predict fine-tuned models on the training data
for model_name, model in best_models.items():
    y_pred = model.predict(X_train)   
    mse = mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    print(f"{model_name} - MSE: {mse}, R^2: {r2}")

#final eval on Test Set for the best model (RandomForestRegressor)
best_rf_model = best_models["RandomForestRegressor"]
best_rf_model.fit(X_train, y_train)
y_test_pred = best_rf_model.predict(X_test)

#calc RMSE and R² on the test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)

print(f"Final Evaluation on Test Set for RandomForestRegressor:")
print(f"Test RMSE: {test_rmse}")
print(f"Test R²: {test_r2}")


Tuning GradientBoostingRegressor...
Fitting 5 folds for each of 27 candidates, totalling 135 fits


Traceback (most recent call last):
  File "/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_comm.py", line 422, in _on_run
    cmd.send(self.sock)
  File "/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_net_command.py", line 111, in send
    sock.sendall(("Content-Length: %s\r\n\r\n" % len(as_bytes)).encode("ascii"))
Traceback (most recent call last):
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_comm.py", line 422, in _on_run
    cmd.send(self.sock)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/debugpy/_vendored/pydevd/_pydevd

Best parameters for GradientBoostingRegressor: {'model__learning_rate': 0.1, 'model__max_depth': 7, 'model__n_estimators': 150}
Tuning RandomForestRegressor...
Fitting 5 folds for each of 24 candidates, totalling 120 fits


/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Traceback (most recent call last):
  File "/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_comm.py", line 422, in _on_run
    cmd.send(self.sock)
  File "/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_net_command.py", line 111, in send
    sock.sendall(("Content-Length: %s\r\n\r\n" % len(as_bytes)).encode("ascii"))
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/sio.c/.pyenv/versions/3.12.0/envs/myproject/lib/python3.12/site-packages/debu

Best parameters for RandomForestRegressor: {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 100}
Tuning Ridge...
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters for Ridge: {'model__alpha': 10}
GradientBoostingRegressor - MSE: 9632.51023737348, R^2: 0.9999998267895188
RandomForestRegressor - MSE: 285.6674194199232, R^2: 0.9999999948631676
Ridge - MSE: 750875077.4659798, R^2: 0.9864978670928948
Final Evaluation on Test Set for RandomForestRegressor:
Test RMSE: 33.15256359393786
Test R²: 0.9999999803980933


In [63]:
#summary table - mean and std of CV RMSE
cv_results_df = pd.DataFrame(cv_results).T  # Convert the dictionary to a DataFrame
cv_results_df = cv_results_df.sort_values(by='Mean CV RMSE', ascending=True)  # Sort by Mean CV RMSE

#print
print("Model Performance Summary:")
print(cv_results_df)

Model Performance Summary:
                           Mean CV RMSE   Std CV RMSE
RandomForestRegressor      5.116004e+01  3.268903e+01
GradientBoostingRegressor  1.200248e+02  1.501250e+01
Ridge                      3.633573e+07  7.257312e+07


### Part 4: Discussion [4 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?
- Provide a ranking of your three models and explain your reasoning — not just based on RMSE, but also interpretability, training time, or generalizability.
- Conclude by considering whether this workflow has produced the results you expected. Typically, you would repeat steps 2 - 4 and also reconsider the choices you made in Milestone 1 when cleaning the dataset, until reaching the point of diminishing returns; do you think that would that have helped here?

#### **What was your tuning strategy for each model? Why did you choose those hyperparameters?**

Tuning Strategy for Each Model:
1. GradientBoostingRegressor:
    * Tuning Strategy:
        * We focused on tuning three main hyperparameters:
            * n_estimators: This controls the number of trees in the model. We tested different values (50, 100, 150) to ensure the model didn’t overfit while still maintaining good performance.
            * learning_rate: This parameter controls how much each tree contributes to the final prediction. We experimented with values (0.01, 0.05, 0.1) to strike a balance between speed and accuracy.
            * max_depth: We tested depths of 3, 5, and 7 to manage tree complexity. If the trees are too deep, we risk overfitting; if they’re too shallow, we might underfit the data.
    * Reason for Choosing These Hyperparameters:
        * These parameters were selected because they directly impact the model's performance and complexity. Our goal was to find the best configuration that minimizes the bias-variance trade-off.
2. RandomForestRegressor:
    * Tuning Strategy:
        * We focused on tuning the following hyperparameters:
            * n_estimators: Similar to GradientBoostingRegressor, we tested various values to identify the optimal number of trees.
            * max_depth: This controls how deep the trees can grow. By tuning this parameter, we balance overfitting and underfitting.
            * min_samples_split and min_samples_leaf: These control how the trees are built by adjusting the minimum number of samples required to split an internal node and to be a leaf. This helps improve generalization and prevent overfitting.
    * Reason for Choosing These Hyperparameters:
        * These parameters are commonly used to balance the flexibility and simplicity of the model. They help control overfitting, especially when using a high number of trees, by ensuring the trees don’t become too complex.
3. Ridge:
    * Tuning Strategy:
        * For Ridge, the key hyperparameter we focused on was:
            * alpha: This parameter controls the strength of regularization. We tested values (0.1, 1, and 10) to determine the optimal level of regularization.
    * Reason for Choosing These Hyperparameters:
        * The alpha parameter is critical in Ridge regression for controlling regularization. A larger value penalizes large coefficients more heavily, preventing overfitting; a smaller avalue allows the model more flexibility to fit the data.



#### **Did you find that certain types of preprocessing or feature engineering worked better with specific models?**

**Preprocessing:**
* Scaling:
    * For Ridge regression, scaling was essential because it’s a linear model that requires normalization to ensure all features contribute equally.
    * RandomForestRegressor and GradientBoostingRegressor, being tree-based models, are not sensitive to scaling, so we skipped this step for them.
* Imputation:
    * We used SimpleImputer to handle missing data, which was a common issue across all models. 

**Feature Engineering:**
* Polynomial Features for Ridge:
    * Adding polynomial features for Ridge helped capture non-linear relationships. This transformation improved Ridge’s performance.
    * However, RandomForestRegressor and GradientBoostingRegressor did not show similar improvements.
* Feature Selection with SelectKBest:
    * Feature selection was effective for RandomForestRegressor and GradientBoostingRegressor. Both models identified the most important features, improving both performance and interpretability.
    * For Ridge, feature selection didn’t significantly impact performance.


#### **Provide a ranking of your three models and explain your reasoning — not just based on RMSE, but also interpretability, training time, or generalizability.**
The following three models were selected and ranked based on Based on their RMSE, interpretability, training time, and generalizability:
1. RandomForestRegressor:
* Mean CV RMSE: 51.16
* Std CV RMSE: 32.69
    * RMSE: This model performed the best, achieving the lowest RMSE. It demonstrates the best predictive accuracy among the three models.
    * Interpretability: Moderately interpretable. Feature importance can be extracted.
    * Training Time: Took 10 minutes in VS Code, but took too long (1.5 hours) in Google Colab. Efficient in VS Code.
    * Generalizability: Excels in generalization due to its ensemble approach. Less prone to overfitting and can handle many features effectively.


2. GradientBoostingRegressor:
* Mean CV RMSE: 120.02
* Std CV RMSE: 15.01
    * RMSE: This model has a higher RMSE than RandomForestRegressor, but it still performs well overall.
    * Interpretability: Less interpretable due to sequential tree-building. Feature importance can still be extracted.
    * Training Time: Slower than RandomForestRegressor due to sequential tree-building.
    * Generalizability: Highly accurate, but more prone to overfitting if not carefully tuned. Requires more training time and tuning.
    


3. Ridge:
* Mean CV RMSE: 36,335,730
* Std CV RMSE: 72,573,120
    * RMSE: This model had the highest RMSE among the three, indicating it struggled with performance on this dataset.
    * Interpretability: Highly interpretable as a linear model with clear coefficients.
    * Training Time: Very fast to train, as it is a linear model.
    * Generalizability: Performs well with linear relationships but struggles with non-linear data, leading to poor performance here.



#### **Conclude by considering whether this workflow has produced the results you expected. Typically, you would repeat steps 2 - 4 and also reconsider the choices you made in Milestone 1 when cleaning the dataset, until reaching the point of diminishing returns; do you think that would that have helped here?**

This workflow has mostly delivered the results we anticipated. Visualizing the mean squared error (MSE) and cross-validation standard deviation proved particularly valuable for diagnosing instances of underfitting or excessive instability in the models.
There is always potential for further improvement. If we were to revisit steps 2–4 or go back to Milestone 1, we believe we could enhance performance even more, such as:
* Repeating steps 1 and 2 with slightly adjusted parameter ranges, informed by insights from previous runs, could lead to the discovery of better-performing models.
* Reevaluating our data cleaning choices—such as our approach to handling missing values, encoding categorical features, or addressing outliers—might enhance generalization.
* Revisiting feature engineering or selection could simplify the models, reduce variance, or improve interpretability.


##### Other note / Warnings and Errors:
During the model tuning process, we encountered warnings and errors related to parallel processing and socket communication. 
There were warnings about "frozen modules" being used, which may cause some issues with debugging. Error “OSError: [Errno 9] Bad file descriptor” was displayed during the cleanup of the worker threads used for parallel processing in GridSearchCV.  The warnings are common when working with parallel computation frameworks and can be ignored in this context; the final model performance should remain unaffected.